<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-14 09:47:51
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 5.77 K (0.04%)
Current PnL: -21.78 L (-14.81%)
CY Booked + Current PnL: -13.51 L (-9.19%)
-------------------
Total profit:  2.59 L
Total loss:  -24.37 L
-------------------
Total Booked + Current PnL: 13.47 L (11.22%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.25 L (61.05%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.22,111096.0,6784.0,9865.0,-0.88,6.50,8.88,15.96,11.0,0.69,0.85,24.50,X5K,ATH,METALS
68,SBIN,760.30,863.00,-11.34,M-LC,3.82,218519.0,17040.0,10183.0,0.33,8.46,4.66,13.51,61.0,1.67,1.67,22.12,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-14.30,H-LC,5.91,171314.0,15137.0,10964.0,-0.28,9.69,6.40,16.71,19.0,1.38,1.31,28.66,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,11.34,H-MC,13.82,159001.0,32801.0,13372.0,1.11,25.99,8.41,36.59,111.0,2.45,1.22,41.76,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-25.14,H-SC,10.15,223604.0,30584.0,17463.0,-0.22,15.84,7.81,24.89,134.0,1.75,1.71,42.89,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.80,H-SC,4.62,101790.0,-710.0,41540.0,0.78,-0.69,40.81,39.83,116.0,-0.02,0.78,0.78,AR,ATH,MISC
25,DABUR,505.20,735.00,-9.60,H-MC,3.50,196774.0,-254.0,89867.0,0.21,-0.13,45.67,45.49,103.0,-0.00,1.51,12.89,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-9.47,H-MC,7.74,189975.0,-1767.0,97799.0,-0.75,-0.92,51.48,50.08,101.0,-0.02,1.45,5.81,XY25,ATH,AC
56,NATIONALUM,189.63,247.44,-50.29,H-MC,1.98,99857.0,-647.0,31285.0,-0.81,-0.64,31.33,30.49,80.0,-0.02,0.76,31.55,MH,ATH,METALS
92,WIPRO,243.46,420.00,-13.79,M-LC,7.37,151565.0,620.0,108839.0,1.16,0.41,71.81,72.51,54.0,0.01,1.16,6.46,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.44,H-LC,2.53,214542.0,10042.0,31366.0,0.32,4.91,14.62,20.25,10.0,0.32,1.64,14.07,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.67,H-LC,3.15,170340.0,-31046.0,118659.0,0.19,-15.42,69.66,43.51,27.0,-0.26,1.30,18.57,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-16.50,H-LC,3.26,128560.0,-136866.0,193778.0,0.17,-51.56,150.73,21.44,8.0,-0.71,0.98,3.31,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-6.07,H-LC,3.72,186300.0,-14010.0,73980.0,1.18,-6.99,39.71,29.94,17.0,-0.19,1.43,28.71,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-11.16,H-LC,5.80,279552.0,9205.0,25411.0,0.04,3.40,9.09,12.81,28.0,0.36,2.14,16.91,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.73,60.62,XR,NTT,CHEMICALS
10,BAJAJHFL,122.26,152.00,-11.52,H-MC,5.29,186238.0,-14268.0,63042.0,4.61,-7.12,33.85,24.33,98.0,-0.23,1.43,7.40,X40N,BTT,FINANCE
29,EASEMYTRIP,18.12,26.40,-8.10,M-SC,24.83,88407.0,-88408.0,169202.0,2.26,-50.00,191.39,45.70,197.0,-0.52,0.68,2.84,XY24,NTT,TRAVEL
73,SONACOMS,578.05,1006.00,-36.95,M-SC,13.30,78531.0,-22628.0,97520.0,1.93,-22.37,124.18,74.03,203.0,-0.23,0.60,10.74,AR,BTT,AUTO
19,CAMS,3643.00,5211.76,-4.50,H-SC,3.86,108522.0,6518.0,37408.0,1.42,6.39,34.47,43.06,123.0,0.17,0.83,24.60,X40N,ATH,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMPUS,294.86,393.00,-30.40,M-SC,2.15,145038.0,-22737.0,78582.0,-2.22,-13.55,54.18,33.28,208.0,-0.29,1.11,12.29,XY24,NTT,FOOTWEAR
89,VBL,492.64,671.64,-11.01,H-LC,10.14,270210.0,5170.0,91142.0,-1.73,1.95,33.73,36.33,5.0,0.06,2.07,15.51,X40N,ATH,FMCG
65,SAIL,130.64,228.00,33.12,M-MC,10.20,210084.0,-14878.0,182542.0,-1.65,-6.61,86.89,74.53,193.0,-0.08,1.61,21.98,XY24,BTT,STEEL
52,LAOPALA,369.40,464.00,104.98,H-SC,2.66,65110.0,-35736.0,61562.0,-1.41,-35.44,94.55,25.61,144.0,-0.58,0.50,20.72,AR,NTT,CERAMICS
6,ASIANTILES,75.41,137.00,6550.00,L-SC,6.90,72973.0,-20837.0,97455.0,-1.40,-22.21,133.55,81.67,270.0,-0.21,0.56,41.32,XR,NTT,CERAMICS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.38,M-SC,10.33,83329.0,-17448.0,17541.0,0.17,-17.31,21.05,0.09,243.0,-0.99,0.64,9.48,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,79.70,H-SC,20.42,113778.0,-28701.0,48219.0,-1.36,-20.14,42.38,13.70,163.0,-0.60,0.87,34.29,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.57,H-SC,3.11,114804.0,-28999.0,55749.0,-0.51,-20.17,48.56,18.60,148.0,-0.52,0.88,26.69,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-69.07,H-SC,2.79,213588.0,-56079.0,92420.0,0.36,-20.80,43.27,13.47,142.0,-0.61,1.64,8.03,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,108.71,M-SC,2.91,136946.0,-37613.0,37646.0,0.47,-21.55,27.49,0.02,221.0,-1.00,1.05,18.75,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.95,237190.0,-58744.0,152655.0,0.15,-19.85,64.36,31.73,2.0,-0.38,1.82,1.26,X40,BTT,IT
45,INFY,1461.46,2275.00,-22.11,H-LC,10.63,255361.0,-6240.0,151863.0,0.18,-2.39,59.47,55.67,4.0,-0.04,1.96,3.57,X40,BTT,IT
89,VBL,492.64,671.64,-11.01,H-LC,10.14,270210.0,5170.0,91142.0,-1.73,1.95,33.73,36.33,5.0,0.06,2.07,15.51,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.71,H-LC,10.44,197847.0,-18672.0,111586.0,-0.28,-8.62,56.40,42.91,7.0,-0.17,1.52,3.61,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.50,H-LC,3.26,128560.0,-136866.0,193778.0,0.17,-51.56,150.73,21.44,8.0,-0.71,0.98,3.31,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,11.34,H-MC,13.82,159001.0,32801.0,13372.0,1.11,25.99,8.41,36.59,111.0,2.45,1.22,41.76,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,11.34,H-MC,13.82,159001.0,32801.0,13372.0,1.11,25.99,8.41,36.59,111.0,2.45,1.22,41.76,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,7.87,H-SC,12.23,128696.0,3399.0,55610.0,-0.92,2.71,43.21,47.09,141.0,0.06,0.99,61.07,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.82,H-MC,6.96,175640.0,10970.0,56995.0,0.03,6.66,32.45,41.28,91.0,0.19,1.35,15.18,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-4.50,H-SC,3.86,108522.0,6518.0,37408.0,1.42,6.39,34.47,43.06,123.0,0.17,0.83,24.60,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.87,166502.0,15589.0,41709.0,-0.24,10.33,25.05,37.97,92.0,0.37,1.28,38.25,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.87,166502.0,15589.0,41709.0,-0.24,10.33,25.05,37.97,92.0,0.37,1.28,38.25,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.09,H-SC,12.04,136475.0,13139.0,120630.0,0.87,10.65,88.39,108.45,119.0,0.11,1.05,35.39,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,7.87,H-SC,12.23,128696.0,3399.0,55610.0,-0.92,2.71,43.21,47.09,141.0,0.06,0.99,61.07,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.70,M-SC,4.26,93375.0,1876.0,76876.0,0.30,2.05,82.33,86.07,219.0,0.02,0.72,35.08,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-13.79,M-LC,7.37,151565.0,620.0,108839.0,1.16,0.41,71.81,72.51,54.0,0.01,1.16,6.46,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.57,72691.0,-40858.0,80905.0,-0.63,-35.98,111.30,35.27,268.0,-0.51,0.56,77.91,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.34,H-SC,15.57,89049.0,-11914.0,106449.0,-1.06,-11.80,119.54,93.63,149.0,-0.11,0.68,32.64,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-19.15,H-SC,13.89,91875.0,-6105.0,27976.0,-0.49,-6.23,30.45,22.32,152.0,-0.22,0.70,33.25,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.73,60.62,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.22,111096.0,6784.0,9865.0,-0.88,6.50,8.88,15.96,11.0,0.69,0.85,24.50,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-25.14,H-SC,10.15,223604.0,30584.0,17463.0,-0.22,15.84,7.81,24.89,134.0,1.75,1.71,42.89,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.87,166502.0,15589.0,41709.0,-0.24,10.33,25.05,37.97,92.0,0.37,1.28,38.25,AR,ATH,AUTO
70,SHALBY,261.39,327.00,954.23,M-SC,6.85,143451.0,-37692.0,83159.0,-0.66,-20.81,57.97,25.10,232.0,-0.45,1.10,14.96,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.57,72691.0,-40858.0,80905.0,-0.63,-35.98,111.30,35.27,268.0,-0.51,0.56,77.91,XR,NTT,HOTELS
89,VBL,492.64,671.64,-11.01,H-LC,10.14,270210.0,5170.0,91142.0,-1.73,1.95,33.73,36.33,5.0,0.06,2.07,15.51,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.39
1,25,41.17
2,50,70.97


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.39
LC    34.31
MC    26.32
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.86
X40N     15.57
X40      13.34
XY25     10.66
XR        9.06
AR        8.45
OX40N     7.51
X5K       2.27
MH        1.75
X200      1.43
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.95
H-SC    24.57
H-MC    23.37
M-SC    13.34
M-LC     7.27
M-MC     2.61
L-SC     1.48
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.22,-10.99,46.66
IT,11.99,-19.18,78.71
FINANCE,8.30,-20.31,64.82
BANKS,7.63,-14.48,63.09
MISC,5.91,-19.32,87.72
PAINTS,5.64,-18.60,41.98
HEALTHCARE,5.34,5.85,17.92
ELECTRICAL,4.74,-13.53,62.69
AUTO,4.49,-15.27,65.41


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2869627.0,23
XR,1111130.0,13
AR,1106991.0,10
X40,829106.0,10
XY25,654904.0,8
X40N,574954.0,12
OX40N,478903.0,10
SR,196470.0,2
X5K,142492.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2808243.0,28
M-SC,1504582.0,17
H-LC,1348349.0,18
H-MC,1326677.0,18
M-LC,429593.0,5
M-MC,336897.0,2
L-SC,268413.0,3
L-MC,58288.0,1
L-LC,44410.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      941982.0      6
M-SC       XY24      876549.0      7
H-SC       AR        672153.0      5
           XR        521823.0      6
H-LC       X40       518098.0      5
H-MC       XY24      495301.0      5
M-MC       XY24      336897.0      2
H-LC       XY25      254703.0      3
           AR        232391.0      2
H-MC       X40       227449.0      4
           XY25      221741.0      2
H-LC       X40N      218358.0      5
H-SC       X40N      213403.0      4
           OX40N     206802.0      4
           SR        196470.0      2
L-SC       XR        178360.0      2
M-LC       XY24      177944.0      2
M-SC       AR        160738.0      2
H-MC       X40N      143193.0      3
M-SC       OX40N     133767.0      4
M-LC       X5K       132627.0      1
M-SC       XR        126102.0      2
           XY25      123867.0      1
H-MC       XR        117718.0      1
M-LC       XR        108839.0      1
L-SC       OX40N      90053.0      1
M-SC       X40        83559.0      1
H-LC       X200       73980.0      1
L-MC       XR         58288.0      1
H-SC       MH         55610.0      1
H-MC       OX40N      48281.0      1
L-LC       XY25       44410.0      1
H-MC       AR         41709.0      1
H-LC       XY24       40954.0      1
H-MC       MH         31285.0      1
M-LC       XY25       10183.0      1
H-LC       X5K         9865.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
